<a href="https://colab.research.google.com/github/WillDera/Stability_of_the_Grid_System/blob/master/Predicting_Stability_Of_Grid_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import sklearn.utils
from sklearn.preprocessing import LabelEncoder
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing

# For our models
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier 
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier 

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# reading our dataset
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv')
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [3]:
# Checking if we have any null values in the dataset
df.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [4]:
# checking the dtypes of the columns in our dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    10000 non-null  float64
 1   tau2    10000 non-null  float64
 2   tau3    10000 non-null  float64
 3   tau4    10000 non-null  float64
 4   p1      10000 non-null  float64
 5   p2      10000 non-null  float64
 6   p3      10000 non-null  float64
 7   p4      10000 non-null  float64
 8   g1      10000 non-null  float64
 9   g2      10000 non-null  float64
 10  g3      10000 non-null  float64
 11  g4      10000 non-null  float64
 12  stab    10000 non-null  float64
 13  stabf   10000 non-null  object 
dtypes: float64(13), object(1)
memory usage: 1.1+ MB


In [5]:
# dropping the stab column and getting our new dataset
df = df.drop(columns="stab", axis=1)
df.head(3)

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable


In [6]:
# encoding the stabf column to 0 and 1 for our model to be able to take in the data.
new_df = df
new_df.stabf = new_df.stabf.map(dict(unstable=1, stable=0))

# Checking our dataset to be sure it looks just how we want it.
new_df.head(2)

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,1
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,0


In [7]:
# preprocessing
X = new_df.drop(columns='stabf', axis=1)
y = new_df["stabf"]

In [9]:
# Standardizing the data
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)

In [10]:
# Splitting our dataset into train and test sets (80% and 20%)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=1)

# **Training Models**

**Random Forest Classifier**

In [11]:
rf_clf = RandomForestClassifier(random_state = 1)
rf_clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

**Extra Trees Classifier**

In [12]:
n_estimators = [50, 100, 300, 500, 1000]

min_samples_split = [2, 3, 5, 7, 9]

min_samples_leaf = [1, 2, 4, 6, 8]

max_features = ['auto', 'sqrt', 'log2', None] 

hyperparameter_grid = {'n_estimators': n_estimators,

                       'min_samples_leaf': min_samples_leaf,

                       'min_samples_split': min_samples_split,

                       'max_features': max_features}

In [13]:
# We're going to use RandomizedSearchCV to find the optimal parameters for our model
from sklearn.model_selection import RandomizedSearchCV

In [15]:
model = ExtraTreesClassifier(random_state=1)
extree_clf = RandomizedSearchCV(estimator=model, param_distributions=hyperparameter_grid)
extree_clf.fit(x_train, y_train)

print("\n The best estimator across ALL searched params:\n", extree_clf.best_estimator_)
print("\n The best score across ALL searched params:", extree_clf.best_score_)
print("\n The best parameters across ALL searched params:\n", extree_clf.best_params_)


 The best estimator across ALL searched params:
 ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=5,
                     min_weight_fraction_leaf=0.0, n_estimators=500,
                     n_jobs=None, oob_score=False, random_state=1, verbose=0,
                     warm_start=False)

 The best score across ALL searched params: 0.928375

 The best parameters across ALL searched params:
 {'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'auto'}


**XGBoost Classifier**

In [16]:
xgbc_clf = XGBClassifier(random_state=1)
xgbc_clf.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

**Lightgbm Classifier**

In [17]:
lgbm_clf = LGBMClassifier(random_state=1)
lgbm_clf.fit(x_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

# **Model Evaluation**

In [28]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score

***Evaluation: Random Forest***

In [29]:
# confusion matrix for our random forest classifier
rf_pred = cross_val_predict(rf_clf, x_train, y_train, cv=10)
cm = confusion_matrix(y_train, rf_pred)
cm

array([[2497,  411],
       [ 229, 4863]])

In [31]:
# getting the precision, recall and f1 scores of our Random Forest classifier
print("Random Forest Classifier: \n")
print("Precision: ", precision_score(y_train, rf_pred))
print("Recall: ", recall_score(y_train, rf_pred))
print("F1-score: ", f1_score(y_train, rf_pred))

Random Forest Classifier: 

Precision:  0.9220705346985211
Recall:  0.9550274941084054
F1-score:  0.9382596951572449


***Evaluation: Extra Trees Classifier***

In [32]:
# confusion matrix for our Extra Trees classifier
extree_pred = cross_val_predict(extree_clf, x_train, y_train, cv=10)
cm = confusion_matrix(y_train, extree_pred)
cm

array([[2501,  407],
       [ 197, 4895]])

In [33]:
# getting the precision, recall and f1 scores of our Extra Trees classifier
print("Extra Trees Classifier: \n")
print("Precision: ", precision_score(y_train, extree_pred))
print("Recall: ", recall_score(y_train, extree_pred))
print("F1-score: ", f1_score(y_train, extree_pred))

Extra Trees Classifier: 

Precision:  0.9232365145228216
Recall:  0.961311861743912
F1-score:  0.9418895516644218


***Evaluation: XGBoost Classifier***

In [34]:
# confusion matrix for our XGBoost classifier
xgbc_pred = cross_val_predict(xgbc_clf, x_train, y_train, cv=10)
cm = confusion_matrix(y_train, xgbc_pred)
cm

array([[2467,  441],
       [ 194, 4898]])

In [35]:
# getting the precision, recall and f1 scores of our XGBoost classifier
print("XGBoost Classifier: \n")
print("Precision: ", precision_score(y_train, xgbc_pred))
print("Recall: ", recall_score(y_train, xgbc_pred))
print("F1-score: ", f1_score(y_train, xgbc_pred))

XGBoost Classifier: 

Precision:  0.9174002622213898
Recall:  0.9619010212097407
F1-score:  0.9391237656983991


***Evaluation: LightGBM Classifier***

In [36]:
# confusion matrix for our LightGBM classifier
lgbm_pred = cross_val_predict(lgbm_clf, x_train, y_train, cv=10)
cm = confusion_matrix(y_train, lgbm_pred)
cm

array([[2634,  274],
       [ 193, 4899]])

In [37]:
# getting the precision, recall and f1 scores of our LightGBM classifier
print("LightGBM Classifier: \n")
print("Precision: ", precision_score(y_train, lgbm_pred))
print("Recall: ", recall_score(y_train, lgbm_pred))
print("F1-score: ", f1_score(y_train, lgbm_pred))

LightGBM Classifier: 

Precision:  0.9470326696307751
Recall:  0.9620974076983504
F1-score:  0.9545056015586946
